# Setting runtime variables dynamically

In [4]:
import { tool } from "@langchain/core/tools"
import { getContextVariable, setContextVariable } from "@langchain/core/context"
import { z } from "npm:zod"

let currency 

const currTool = tool(

    async (input) => {
        
        const generatedCurr = getContextVariable("currency")
        if (!input.currName) return "No curr!!!!!!!!!!!"
        
        currency = generatedCurr
        return "currency_tool called"
    },
    {
        name: "currency_tool",
        description: "gets set currency info.",
        schema: z.object({
            currName: z.string().min(6).max(6)
        })
    })

await currTool.invoke({ currName: "USDCAD" })

currency ? currency : "N/A"

"N/A"

In [6]:
import { BaseChatModel } from "@langchain/core/language_models/chat_models"
import { RunnableLambda } from "@langchain/core/runnables"
import { ChatPromptTemplate } from "@langchain/core/prompts"
import { StringOutputParser } from "@langchain/core/output_parsers"
import { ChatGroq } from "@langchain/groq"


const genCurr = async ({ llm, ui }: { llm: BaseChatModel, ui: string }) => {
    try {
        const tooledLLM = llm.bindTools([currTool])

        const prompt = ChatPromptTemplate.fromTemplate("You are an expert trader who who knows structure of currency pairs. Generate an uppercase currency pair name of 6 characters in the format ABCXYZ from the user input: {user_input}.")
        const parser = new StringOutputParser()
        const chain = prompt.pipe(llm).pipe(parser).pipe(tooledLLM)

        const modelResponse = await chain.invoke({ user_input: ui })

        const toolCallOutPut = modelResponse?.tool_calls[0]

        const currName = toolCallOutPut?.args?.currName
        setContextVariable("currency", currName)

        return currTool.invoke(toolCallOutPut)
    } catch (error) {
        return error.message
    }
}

const genCurrRunnable = RunnableLambda.from(genCurr)

await genCurrRunnable.invoke({
    llm: new ChatGroq(),
    ui: "I trade gold American currency"
})

currency ? currency : "N/A"

"XAUUSD"